In [48]:
import pandas as pd 
import numpy as np 
transactions = pd.read_csv("data/transactions.csv")
transactions["day"] = pd.to_datetime(transactions["day"], dayfirst=False)

## Preparing data 
Filter on a single product for now, preparing a dataframe that can be used to fit a demand function

In [49]:
product = "Oreo"

filtered_transactions = transactions[(transactions["description"] == product)]
#filtered_transactions["hour"] = pd.to_timedelta(filtered_transactions["time"])

def include_hour(row):
    hour = pd.to_datetime(row["time"]).hour
    return hour
filtered_transactions["hour"] = filtered_transactions.apply(include_hour, axis = 1)
filtered_transactions

<ipython-input-49-dc53a50744c4>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic,hour
76,2018-01-01,10:27:48,5.0,50330345.0,snack,s_3,Oreo,1st,1.29,1.29,0.0,0.0,10.0
81,2018-01-01,10:27:48,5.0,50330345.0,snack,s_3,Oreo,1st,1.29,1.29,0.0,0.0,10.0
101,2018-01-01,10:44:42,7.0,56401558.0,snack,s_3,Oreo,1st,1.29,1.29,0.0,0.0,10.0
109,2018-01-01,10:44:42,7.0,56401558.0,snack,s_3,Oreo,1st,1.29,1.29,0.0,0.0,10.0
127,2018-01-01,11:07:01,9.0,NaN,snack,s_3,Oreo,1st,1.29,1.29,0.0,0.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
354509,2018-12-30,17:15:08,20669.0,18074226.0,snack,s_3,Oreo,1st,1.29,1.29,0.0,0.0,17.0
354523,2018-12-30,17:16:52,20670.0,22703949.0,snack,s_3,Oreo,1st,1.29,1.29,0.0,0.0,17.0
354762,2018-12-30,19:38:35,20681.0,51945522.0,snack,s_3,Oreo,1st,1.29,1.29,0.0,0.0,19.0
354780,2018-12-30,19:59:18,20682.0,NaN,snack,s_3,Oreo,1st,1.29,1.29,0.0,0.0,19.0


In [50]:
purchases = filtered_transactions.groupby(["day", "hour"]).describe()["purchase_price"]
purchases

count  mean  std   min   25%   50%   75%   max
day        hour                                                
2018-01-01 10.0    4.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29
           11.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29
           13.0    6.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29
           14.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29
           15.0    5.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29
...                ...   ...  ...   ...   ...   ...   ...   ...
2018-12-30 15.0    5.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29
           16.0    1.0  1.29  NaN  1.29  1.29  1.29  1.29  1.29
           17.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29
           19.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29
           20.0    1.0  1.29  NaN  1.29  1.29  1.29  1.29  1.29

[3659 rows x 8 columns]

In [53]:
days = [purchases.index[i][0].dayofyear for i in range(purchases.shape[0])]
hours = [purchases.index[i][1] for i in range(purchases.shape[0])]
purchases["hour"] = hours
purchases["day"] = days
pred_cols = ["min", "max", "hour", "day"]
target_col = "count"
purchases

count  mean  std   min   25%   50%   75%   max  hour  day
day        hour                                                           
2018-01-01 10.0    4.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  10.0    1
           11.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  11.0    1
           13.0    6.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  13.0    1
           14.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  14.0    1
           15.0    5.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  15.0    1
...                ...   ...  ...   ...   ...   ...   ...   ...   ...  ...
2018-12-30 15.0    5.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  15.0  364
           16.0    1.0  1.29  NaN  1.29  1.29  1.29  1.29  1.29  16.0  364
           17.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  17.0  364
           19.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  19.0  364
           20.0    1.0  1.29  NaN  1.29  1.29  1.29  1.29  1.29  20.0  364

[3659 rows x 10 columns]

## Implementing a ML model for demand function

In [65]:
from sklearn.model_selection import train_test_split


train, test = train_test_split(purchases)
train.dropna(inplace=True)
test.dropna(inplace=True)
X_train = train[pred_cols].values
y_train = train[target_col].values

X_test = test[pred_cols].values
y_test = test[target_col].values

<ipython-input-65-8481154c4a32>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-65-8481154c4a32>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



True

In [66]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train, y_train)
linreg.score(X_test, y_test)

0.03224311938423663

In [67]:
purchases = filtered_transactions.groupby("day").describe()["purchase_price"]

import plotly.express as px

fig = px.scatter(purchases, y="count", x=purchases.index, color="mean")
fig.show()

In [30]:
purchases[purchases["std"] > 0.001]

,count,mean,std,min,25%,50%,75%,max
day,,,,,,,,
2018-02-04,9.0,4.155556,0.283333,3.40,4.25,4.25,4.25,4.25
2018-03-26,9.0,3.494444,0.283333,3.40,3.40,3.40,3.40,4.25
2018-04-06,9.0,2.925556,0.496667,2.76,2.76,2.76,2.76,4.25
2018-05-14,8.0,3.690000,0.226274,3.61,3.61,3.61,3.61,4.25
2018-05-21,10.0,4.122000,0.269848,3.61,4.25,4.25,4.25,4.25
2018-06-08,11.0,3.086364,0.385935,2.97,2.97,2.97,2.97,4.25
2018-06-18,6.0,4.075000,0.085732,4.04,4.04,4.04,4.04,4.25
2018-08-13,6.0,4.036667,0.522558,2.97,4.25,4.25,4.25,4.25
2018-10-12,5.0,4.082000,0.093915,4.04,4.04,4.04,4.04,4.25
